In [1]:
# DPLL

Code hieronder is puur om de literal te kiezen die het meeste voorkomt 

In [ ]:
from collections import defaultdict 

def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1
            
    mostLiterals = max(counts, key=counts.get)
    amount = counts[mostLiterals]
    #print("Most occurring literal:", mostLiterals, "with amount:", amount)
    return mostLiterals


# Mijn versie van DPLL
Mocht je willen kijken hoe mijn versie van DPLL eruit ziet


In [ ]:
from typing import List, Tuple, Dict, Optional, Iterable
from collections import defaultdict, deque
import time

Assignment = Dict[int, bool]

def parse_dimacs(input_path: str) -> Tuple[Iterable[Iterable[int]], int]:
    close = False
    if isinstance(input_path, str):
        file = open(input_path, "r")
        close = True
    else:
        file = input_path


    line = file.readline()

    components = line.strip().split(" ")

    if len(components)!= 4 or components[0]!="p" or components[1]!="cnf":
      print("Wrong file format! Expected first line to be 'p cnf NUM_VARS NUM_CLAUSES")
      exit(1)

    num_vars=int(components[2])
    num_clauses=int(components[3])

    clauses=[]

    line=file.readline()
    while(line):
       numbers = [int(x) for x in line.strip().split(" ")]

       if(numbers[-1]!=0):
          print("Wrong format! Clause lines must be terminated with a 0")

       clauses.append(numbers[:-1])

       line=file.readline()


    return clauses, num_vars


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()


def delete_tautologies(clause: list[int], clauses: list[list[int]]) -> int:
    taut_count = 0
    for literal in clause:
        opposite_literal = literal * -1
        if opposite_literal in clause:
            # delete clause from clause list
            #clauses.remove(clause)
            taut_count += 1
            return taut_count
    return taut_count

def pure_literal(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], list[int]]:
    # Nog veel werk aan de winkel
    truth_list = []

    for literal in clause:
        pure = True
        neg_lit = literal * -1
        # Check if only negative or positive in other clauses
        for loop_clause in clauses:
            # Finding prove of literal not being pure
            if neg_lit in loop_clause:
                pure = False
                break
        # Take literal out of all clauses if it is pure
        if pure:
            # print(f"literal {literal} is seen as pure")
            if not literal in truth_list:
                truth_list.append(literal)
            for loop_clause in clauses[:]:
                if literal in loop_clause:
                    clauses.remove(loop_clause)

    return clauses, truth_list



def unit_clause(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], int, int]:
    if len(clause) == 1 and not clause[0] == 0:
        truth = clause[0]
        #clauses.remove(clause)
        """
        for clause_loop in clauses[:]:
            opposite = truth * -1
            if truth in clause_loop:
                clauses.remove(clause_loop)
            elif opposite in clause_loop:
                clause_loop.remove(opposite)        
        """
        new_clauses = []
        opposite = -truth

        for clause in clauses:
            if truth in clause:
                continue  # deze clause is voldaan, dus overslaan
            if opposite in clause:
                clause = [lit for lit in clause if lit != opposite]
            new_clauses.append(clause)

        clauses = new_clauses

        # Take clause out of all clauses
        """for clause_loop in clauses:
            if truth in clause_loop:
                clause_loop.remove(truth)
            elif truth * -1 in clause_loop:
                clause_loop.remove(truth * -1)
        """
        unit = 1
    else:
        unit = 0
        truth = 0
    return clauses, truth, unit



def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    """
    Snelle simplificatie:
      - verwijder tautologieën
      - unit propagation (herhaald, met queue)
      - pure literal elimination (globaal per iteratie)
    Geeft (nieuwe_clauses, truth_list) terug.
    """

    # verzameling van alle geforceerde waar-literals
    truth_set = set()

    # we werken op een lokale kop zodat het origineel niet stuk gaat
    clauses = [list(cl) for cl in clauses]

    changed = True
    while changed:
        changed = False

        # -------------------------
        # 1. Tautologieën + directe unit clauses
        # -------------------------
        unit_queue = deque()
        new_clauses: List[List[int]] = []

        for clause in clauses:
            # duplicates binnen clausule verwijderen
            lits = set(clause)

            # tautologie? (x en -x in dezelfde clausule)
            if any(-lit in lits for lit in lits):
                # sla deze clausule over
                changed = True
                continue

            # unit clausule?
            if len(lits) == 1:
                lit = next(iter(lits))
                unit_queue.append(lit)
                changed = True
                continue

            new_clauses.append(list(lits))

        clauses = new_clauses

        # -------------------------
        # 2. Unit propagation (met queue)
        # -------------------------
        while unit_queue and clauses:
            u = unit_queue.popleft()

            # conflict: zowel u als -u?
            if -u in truth_set:
                # formule is inconsistent -> leeg clause toevoegen
                return ([[]], list(truth_set))

            if u in truth_set:
                continue  # al verwerkt

            truth_set.add(u)
            changed = True

            new_clauses = []
            for clause in clauses:
                # clause is voldaan?
                if u in clause:
                    continue

                # tegensgestelde literal verwijderen
                if -u in clause:
                    new_clause = [lit for lit in clause if lit != -u]
                    if len(new_clause) == 0:
                        # lege clausule -> UNSAT signaal voor DPLL
                        new_clauses.append(new_clause)
                    elif len(new_clause) == 1:
                        # nieuwe unit literal gevonden
                        unit_queue.append(new_clause[0])
                        new_clauses.append(new_clause)
                    else:
                        new_clauses.append(new_clause)
                else:
                    new_clauses.append(clause)

            clauses = new_clauses

        # Als we al een lege clausule hebben, hoeven we niet verder
        if any(len(cl) == 0 for cl in clauses):
            break

        # -------------------------
        # 3. Pure literal elimination
        # -------------------------
        # tel voorkomen van iedere literal
        lit_count = defaultdict(int)
        for clause in clauses:
            for lit in clause:
                lit_count[lit] += 1

        # pure literals = lit met count>0 en count(-lit)==0
        pure_lits = [
            lit for lit, cnt in lit_count.items()
            if cnt > 0 and lit_count.get(-lit, 0) == 0
        ]

        if pure_lits:
            changed = True
            pure_set = set(pure_lits)
            # voeg toe aan assignment
            for p in pure_set:
                truth_set.add(p)

            # alle clausules die een pure literal bevatten zijn voldaan -> weg
            new_clauses = []
            for clause in clauses:
                if any(lit in pure_set for lit in clause):
                    continue
                new_clauses.append(clause)
            clauses = new_clauses

    # einde while: geen veranderingen meer
    return clauses, list(truth_set)


##################################################################################
# DPLL
##################################################################################

DPLL_CALLS = 0


def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1
            
    mostLiterals = max(counts, key=counts.get)
    amount = counts[mostLiterals]
    #print("Most occurring literal:", mostLiterals, "with amount:", amount)
    return mostLiterals

def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:
    
    # Check how many times DPLL is called
    global DPLL_CALLS
    DPLL_CALLS += 1
    print("DPLL call number:", DPLL_CALLS)
    if assignment is None:
        assignment = []

    # Simplify the clauses
    simplified_clauses, implied_truths = simplify(clauses)

    # Add implied truths to assignment
    full_assignment = assignment.copy()
    for literal in implied_truths:
        if -literal in full_assignment:
            # conflict in assignment
            return False, []
        if literal not in full_assignment:
            full_assignment.append(literal)

    # Check for empty clause set -> return UNSAT
    if any(len(cl) == 0 for cl in simplified_clauses):
        return False, []

    # Check if all clauses are satisfied
    if not simplified_clauses:
        return True, full_assignment

    # DLIS step -> choose literal to split on (assign true or false)        
    split_lit = choose_literal_dlis(simplified_clauses)

    sat, model = dpll(
    simplified_clauses + [[split_lit]],
    num_vars,
    full_assignment + [split_lit])

    if sat:
        return True, model

    sat,model = dpll(
    simplified_clauses + [[-split_lit]],
    num_vars,
    full_assignment + [-split_lit])

    return sat, model 


if __name__ == "__main__":
    # clauses, num_vars = parse_dimacs("inst16.cnf")
    # choose_literal_dlis(clauses)
    # sat, assignment = dpll(clauses, num_vars)
    # if sat:
    #     print("SAT")
    #     print(assignment)   
    # else:
    #     print("UNSAT")
    clauses, num_vars = parse_dimacs("inst16.cnf")

    import time
    start = time.perf_counter()
    sat, assignment = dpll(clauses, num_vars)
    end = time.perf_counter()

    print("SAT:", sat)
    print("DPLL calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")  

1 2 3 4 5 6 7 8 9 0


# Fast Solver
versie waarbij simplify verkort is en JW heuristic

In [ ]:
from typing import List, Tuple, Dict, Optional, Iterable
from collections import defaultdict, deque
import time

Assignment = Dict[int, bool]

##################################################################################
# DIMACS I/O (zoals je al had)
##################################################################################

def parse_dimacs(input_path: str) -> Tuple[Iterable[Iterable[int]], int]:
    close = False
    if isinstance(input_path, str):
        file = open(input_path, "r")
        close = True
    else:
        file = input_path

    # lees header
    line = file.readline()
    while line and line.startswith("c"):
        # commentregels overslaan
        line = file.readline()

    components = line.strip().split()

    if len(components) != 4 or components[0] != "p" or components[1] != "cnf":
        print("Wrong file format! Expected first line to be 'p cnf NUM_VARS NUM_CLAUSES'")
        exit(1)

    num_vars = int(components[2])
    num_clauses = int(components[3])

    clauses = []

    line = file.readline()
    while line:
        # sla commentregels over
        if line.startswith("c") or not line.strip():
            line = file.readline()
            continue

        numbers = [int(x) for x in line.strip().split()]
        # DIMACS clausules kunnen over meerdere regels lopen, maar in jullie geval
        # lijkt het per regel één clausule + 0. Dit is de simpele variant:
        if numbers[-1] != 0:
            print("Wrong format! Clause lines must be terminated with a 0")
            exit(1)

        clauses.append(numbers[:-1])
        line = file.readline()

    if close:
        file.close()

    return clauses, num_vars


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()

##################################################################################
# Snelle simplify: alleen unit propagation + tautologieën weggooien
##################################################################################

def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    """
    Snelle simplificatie:
      - verwijder tautologieën
      - verzamel unit clauses
      - unit propagation met een queue

    GEEN pure literal elimination in de recursie (scheelt enorm veel tijd).

    Geeft (nieuwe_clauses, implied_literals) terug.
    """

    truth_set = set()
    unit_queue = deque()

    new_clauses: List[List[int]] = []

    # Eerste pass: duplicates eruit, tautologieën weg, units verzamelen
    for clause in clauses:
        if not clause:
            # lege clause => direct conflict
            return [[]], list(truth_set)

        seen = set()
        lits = []
        taut = False

        for lit in clause:
            if -lit in seen:
                # tautologie (x en -x in dezelfde clause) => altijd waar, skip
                taut = True
                break
            if lit not in seen:
                seen.add(lit)
                lits.append(lit)

        if taut:
            continue  # clause is altijd waar, hoeven we niet mee te nemen

        if len(lits) == 0:
            # lege clause => conflict
            return [[]], list(truth_set)
        elif len(lits) == 1:
            # unit clause
            unit_queue.append(lits[0])
        else:
            new_clauses.append(lits)

    clauses = new_clauses

    # Unit propagation
    while unit_queue and clauses:
        u = unit_queue.popleft()

        # conflict met eerdere literal?
        if -u in truth_set:
            return [[]], list(truth_set)

        if u in truth_set:
            continue  # al verwerkt

        truth_set.add(u)

        new_clauses = []
        for clause in clauses:
            # clause is voldaan?
            if u in clause:
                continue

            # verwijder tegensgestelde literal
            if -u in clause:
                reduced = [lit for lit in clause if lit != -u]
                if len(reduced) == 0:
                    # lege clause => conflict
                    return [[]], list(truth_set)
                if len(reduced) == 1:
                    # nieuwe unit
                    unit_queue.append(reduced[0])
                new_clauses.append(reduced)
            else:
                new_clauses.append(clause)

        clauses = new_clauses

    return clauses, list(truth_set)

##################################################################################
# Heuristiek: Jeroslow–Wang (JW)
##################################################################################

def choose_literal_jw(clauses: List[List[int]]) -> int:
    """
    Jeroslow–Wang heuristiek:

      score(l) = som over clausules C die l bevatten van 2^(-|C|)

    Korte clausules wegen zwaarder; dat helpt vaak sterk bij SAT-instances (zoals Sudoku).
    """
    scores: Dict[int, float] = defaultdict(float)

    for clause in clauses:
        # gewicht van de clause (korte clausules krijgen meer gewicht)
        w = 2.0 ** (-len(clause))
        for lit in clause:
            scores[lit] += w

    if not scores:
        # zou eigenlijk niet mogen gebeuren als we hier komen,
        # maar voor de zekerheid:
        return 0

    return max(scores, key=scores.get)

##################################################################################
# DPLL
##################################################################################

DPLL_CALLS = 0

def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:
    """
    Klassieke DPLL:
      - simplify (unit propagation)
      - check SAT/UNSAT
      - kies literal via JW
      - branch: lit = True, daarna lit = False

    Geen clause learning, geen watched literals (nog), wel veel lichter dan je vorige versie.
    """

    global DPLL_CALLS
    DPLL_CALLS += 1
    if DPLL_CALLS % 100000 == 0:
      print("DPLL call number:", DPLL_CALLS)
      
    if assignment is None:
        assignment = []

    # Simplify met unit propagation
    simplified_clauses, implied_truths = simplify(clauses)

    # Voeg implied literals toe aan assignment
    full_assignment = assignment.copy()
    assigned_set = set(full_assignment)

    for lit in implied_truths:
        if -lit in assigned_set:
            # conflict in assignment
            return False, []
        if lit not in assigned_set:
            full_assignment.append(lit)
            assigned_set.add(lit)

    # Check op conflict: lege clause
    if any(len(cl) == 0 for cl in simplified_clauses):
        return False, []

    # Geen clausules meer => alle zijn voldaan
    if not simplified_clauses:
        return True, full_assignment

    # Kies splitsliteral met Jeroslow–Wang
    split_lit = choose_literal_jw(simplified_clauses)
    if split_lit == 0:
        # fallback: als er toch iets misging
        split_lit = simplified_clauses[0][0]

    # Branch 1: split_lit = True
    sat, model = dpll(
        simplified_clauses + [[split_lit]],
        num_vars,
        full_assignment + [split_lit]
    )
    if sat:
        return True, model

    # Branch 2: split_lit = False
    sat, model = dpll(
        simplified_clauses + [[-split_lit]],
        num_vars,
        full_assignment + [-split_lit]
    )
    return sat, model

##################################################################################
# Main (optioneel testje op een CNF-file)
##################################################################################

if __name__ == "__main__":
    # Voorbeeld: rechtstreeks een .cnf bestand draaien
    # Pas dit pad aan naar jouw gegenereerde DIMACS file (bijv. enc16.cnf)
    cnf_path = "enc16.cnf"

    clauses, num_vars = parse_dimacs(cnf_path)

    start = time.perf_counter()
    sat, assignment = dpll(clauses, num_vars)
    end = time.perf_counter()

    print("SAT:", sat)
    print("DPLL calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")

    if sat:
        # Optioneel: assignment sorteren en afdrukken
        assignment = sorted(assignment, key=lambda x: abs(x))
        print("Model (eerste 50 lits):", assignment[:50])


# DLIS en JW gecombineerde heuristic
Lijkt snel te runnen, sneller dan bovenstaande code


In [ ]:
from typing import List, Tuple, Dict, Optional, Iterable
from collections import defaultdict, deque
import time

Assignment = Dict[int, bool]

def parse_dimacs(input_path: str) -> Tuple[Iterable[Iterable[int]], int]:
    close = False
    if isinstance(input_path, str):
        file = open(input_path, "r")
        close = True
    else:
        file = input_path


    line = file.readline()

    components = line.strip().split(" ")

    if len(components)!= 4 or components[0]!="p" or components[1]!="cnf":
      print("Wrong file format! Expected first line to be 'p cnf NUM_VARS NUM_CLAUSES")
      exit(1)

    num_vars=int(components[2])
    num_clauses=int(components[3])

    clauses=[]

    line=file.readline()
    while(line):
       numbers = [int(x) for x in line.strip().split(" ")]

       if(numbers[-1]!=0):
          print("Wrong format! Clause lines must be terminated with a 0")

       clauses.append(numbers[:-1])

       line=file.readline()


    return clauses, num_vars


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()


def delete_tautologies(clause: list[int], clauses: list[list[int]]) -> int:
    taut_count = 0
    for literal in clause:
        opposite_literal = literal * -1
        if opposite_literal in clause:
            # delete clause from clause list
            #clauses.remove(clause)
            taut_count += 1
            return taut_count
    return taut_count

def pure_literal(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], list[int]]:
    # Nog veel werk aan de winkel
    truth_list = []

    for literal in clause:
        pure = True
        neg_lit = literal * -1
        # Check if only negative or positive in other clauses
        for loop_clause in clauses:
            # Finding prove of literal not being pure
            if neg_lit in loop_clause:
                pure = False
                break
        # Take literal out of all clauses if it is pure
        if pure:
            # print(f"literal {literal} is seen as pure")
            if not literal in truth_list:
                truth_list.append(literal)
            for loop_clause in clauses[:]:
                if literal in loop_clause:
                    clauses.remove(loop_clause)

    return clauses, truth_list



def unit_clause(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], int, int]:
    if len(clause) == 1 and not clause[0] == 0:
        truth = clause[0]
        #clauses.remove(clause)
        """
        for clause_loop in clauses[:]:
            opposite = truth * -1
            if truth in clause_loop:
                clauses.remove(clause_loop)
            elif opposite in clause_loop:
                clause_loop.remove(opposite)        
        """
        new_clauses = []
        opposite = -truth

        for clause in clauses:
            if truth in clause:
                continue  # deze clause is voldaan, dus overslaan
            if opposite in clause:
                clause = [lit for lit in clause if lit != opposite]
            new_clauses.append(clause)

        clauses = new_clauses

        # Take clause out of all clauses
        """for clause_loop in clauses:
            if truth in clause_loop:
                clause_loop.remove(truth)
            elif truth * -1 in clause_loop:
                clause_loop.remove(truth * -1)
        """
        unit = 1
    else:
        unit = 0
        truth = 0
    return clauses, truth, unit



def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    """
    Snellere simplificatie:
      - unit propagation (herhaald, met queue)
    Geen tautologie-check, geen pure literals (voor Sudoku meestal zinloos).

    Returns:
        (nieuwe_clauses, truth_list)
        - nieuwe_clauses: vereenvoudigde formule
        - truth_list: alle literals die geforceerd True zijn
    """

    truth_set = set()
    truth_list: List[int] = []

    # Werk op een kopie zodat het origineel intact blijft
    clauses = [cl[:] for cl in clauses]

    # Start: zoek alle unit clauses
    unit_queue = deque()
    for clause in clauses:
        if len(clause) == 1:
            unit_queue.append(clause[0])

    # Unit propagation loop
    while unit_queue and clauses:
        u = unit_queue.popleft()

        # Als -u al geforceerd is -> conflict
        if -u in truth_set:
            # Representatie van UNSAT: formule met lege clause
            return ([[]], truth_list)

        # Als u al verwerkt is, overslaan
        if u in truth_set:
            continue

        truth_set.add(u)
        truth_list.append(u)

        new_clauses: List[List[int]] = []
        for clause in clauses:
            # Clause is voldaan als u erin zit
            if u in clause:
                continue

            # Als -u erin zit, verwijder die literal
            if -u in clause:
                new_clause = [lit for lit in clause if lit != -u]

                if len(new_clause) == 0:
                    # lege clause -> UNSAT
                    return ([[]], truth_list)

                if len(new_clause) == 1:
                    # nieuwe unit literal
                    unit_queue.append(new_clause[0])

                new_clauses.append(new_clause)
            else:
                new_clauses.append(clause)

        clauses = new_clauses

    # Geen nieuwe units meer
    return clauses, truth_list


##################################################################################
# DPLL
##################################################################################

DPLL_CALLS = 0

def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1

    # DLIS: kies literal (met teken) met hoogste count
    most_literal = max(counts, key=counts.get)
    return most_literal

def choose_literal_dlis_jw(clauses: List[List[int]]) -> int:
    dlis_counts = defaultdict(int)
    jw_score = defaultdict(float)

    for clause in clauses:
        weight = 2.0 ** (-len(clause))   # Jeroslow–Wang factor
        for lit in clause:
            dlis_counts[lit] += 1
            jw_score[lit] += weight

    # Combineer beide heuristieken
    combined = {
        lit: dlis_counts[lit] + jw_score[lit] + dlis_counts[lit] * jw_score[lit]
        for lit in dlis_counts.keys()
    }

    return max(combined, key=combined.get)


def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:

    global DPLL_CALLS
    DPLL_CALLS += 1
    if DPLL_CALLS % 100000 == 0:
        print("DPLL call number:", DPLL_CALLS)

    if assignment is None:
        assignment = []

    # 1. Simplify
    simplified_clauses, implied_truths = simplify(clauses)

    # 2. Implied truths toevoegen aan assignment
    full_assignment = assignment.copy()
    for literal in implied_truths:
        if -literal in full_assignment:
            # conflict in assignment
            return False, []
        if literal not in full_assignment:
            full_assignment.append(literal)

    # 3. UNSAT: lege clause aanwezig?
    if any(len(cl) == 0 for cl in simplified_clauses):
        return False, []

    # 4. SAT: geen clausules meer
    if not simplified_clauses:
        return True, full_assignment

    # 5. DLIS: kies literal (met teken)
    split_lit = choose_literal_dlis_jw(simplified_clauses)

    # 6. Eerste tak: literal = True
    sat, model = dpll(
        simplified_clauses + [[split_lit]],
        num_vars,
        full_assignment + [split_lit]
    )
    if sat:
        return True, model

    # 7. Tweede tak: literal = False
    sat, model = dpll(
        simplified_clauses + [[-split_lit]],
        num_vars,
        full_assignment + [-split_lit]
    )

    return sat, model


if _name_ == "_main_":
    # clauses, num_vars = parse_dimacs("inst16.cnf")
    # choose_literal_dlis(clauses)
    # sat, assignment = dpll(clauses, num_vars)
    # if sat:
    #     print("SAT")
    #     print(assignment)   
    # else:
    #     print("UNSAT")
    clauses, num_vars = parse_dimacs("inst16.cnf")

    import time
    start = time.perf_counter()
    sat, assignment = dpll(clauses, num_vars)
    end = time.perf_counter()

    print("SAT:", sat)
    print("DPLL calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")

In [ ]:
from typing import List, Tuple, Optional
from collections import defaultdict, deque
import time


################################################################################
# DIMACS PARSER
################################################################################

def parse_dimacs(path: str) -> Tuple[List[List[int]], int]:
    with open(path, "r") as f:
        # Skip comments
        line = f.readline()
        while line.startswith("c") or line.strip() == "":
            line = f.readline()

        parts = line.split()
        if len(parts) != 4 or parts[0] != "p" or parts[1] != "cnf":
            raise ValueError("Invalid DIMACS header")

        num_vars = int(parts[2])

        clauses = []
        for line in f:
            if not line.strip() or line.startswith("c"):
                continue
            nums = [int(x) for x in line.split()]
            if nums[-1] != 0:
                raise ValueError("Clause must end with 0")
            clauses.append(nums[:-1])

        return clauses, num_vars


################################################################################
# PURE LITERAL ELIMINATION (ONE-TIME PREPROCESS)
################################################################################

def pure_literal_elimination(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    assignment = []
    changed = True

    while changed:
        changed = False

        lit_count = defaultdict(int)
        for clause in clauses:
            for lit in clause:
                lit_count[lit] += 1

        pure = [
            lit for lit in lit_count
            if lit_count[lit] > 0 and lit_count.get(-lit, 0) == 0
        ]

        if not pure:
            break

        changed = True
        pure_set = set(pure)
        assignment.extend(pure)

        new_clauses = []
        for c in clauses:
            if any(l in pure_set for l in c):
                continue
            new_clauses.append(c)
        clauses = new_clauses

    return clauses, assignment


################################################################################
# UNIT PROPAGATION (VERY FAST)
################################################################################

def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    truth_set = set()
    truth_list = []

    # Init unit queue
    unit_queue = deque(lit for c in clauses if len(c) == 1 for lit in c)

    while unit_queue and clauses:
        u = unit_queue.popleft()

        if -u in truth_set:
            return ([[]], truth_list)  # UNSAT

        if u in truth_set:
            continue

        truth_set.add(u)
        truth_list.append(u)

        new_clauses = []
        for clause in clauses:

            # Clause satisfied?
            if u in clause:
                continue

            # Clause contains negation?
            if -u in clause:
                reduced = []
                for lit in clause:
                    if lit != -u:
                        reduced.append(lit)

                if not reduced:
                    return ([[]], truth_list)  # conflict

                if len(reduced) == 1:
                    unit_queue.append(reduced[0])

                new_clauses.append(reduced)
            else:
                new_clauses.append(clause)

        clauses = new_clauses

    return clauses, truth_list


################################################################################
# FAST DLIS HEURISTIC
################################################################################

def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts = defaultdict(int)
    for clause in clauses:
        for lit in clause:
            counts[lit] += 1

    return max(counts, key=counts.get)


################################################################################
# DPLL CORE
################################################################################

DPLL_CALLS = 0

def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:

    global DPLL_CALLS
    DPLL_CALLS += 1

    if DPLL_CALLS % 100000 == 0:
        print("DPLL calls:", DPLL_CALLS)

    if assignment is None:
        assignment = []

    # Simplify formula
    clauses, implied = simplify(clauses)

    # Merge assignments
    full_assignment = assignment.copy()
    assigned = set(full_assignment)

    for lit in implied:
        if -lit in assigned:
            return False, []
        if lit not in assigned:
            assigned.add(lit)
            full_assignment.append(lit)

    # UNSAT? Empty clause
    if any(len(c) == 0 for c in clauses):
        return False, []

    # SAT? No clauses left
    if not clauses:
        return True, full_assignment

    # Choose split literal
    split = choose_literal_dlis(clauses)

    # Branch 1
    sat, model = dpll(clauses + [[split]], num_vars, full_assignment + [split])
    if sat:
        return True, model

    # Branch 2
    return dpll(clauses + [[-split]], num_vars, full_assignment + [-split])


################################################################################
# MAIN TEST
################################################################################

if __name__ == "__main__":
    clauses, num_vars = parse_dimacs("enc16.cnf")

    # One-time pure literal elimination
    clauses, pre_assign = pure_literal_elimination(clauses)

    start = time.perf_counter()
    sat, model = dpll(clauses, num_vars, pre_assign)
    end = time.perf_counter()

    print("SAT:", sat)
    print("Calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")
    if sat:
        print("Example assignment:", sorted(model)[:50])


#  genereren van een  sudoku met % van c random gevulde cellen

In [3]:
from typing import List

def generate_empty_sudoku(filled_percentage: float) -> List[List[int]]:
    """
    Generate an empty Sudoku grid with a certain percentage of cells filled randomly.

    Args:
        filled_percentage (float): Percentage of cells to fill (0.0 to 1.0).

    Returns:
        List[List[int]]: A 9x9 Sudoku grid with some cells filled.
    """
    import random

    # Create an empty 9x9 grid
    sudoku_grid = [[0 for _ in range(9)] for _ in range(9)]

    # Calculate the number of cells to fill
    total_cells = 81
    cells_to_fill = int(total_cells * filled_percentage)

    filled_positions = set()

    while len(filled_positions) < cells_to_fill:
        row = random.randint(0, 8)
        col = random.randint(0, 8)
        if (row, col) not in filled_positions:
            # Fill the cell with a random number between 1 and 9
            sudoku_grid[row][col] = random.randint(1, 9)
            filled_positions.add((row, col))

    return sudoku_grid

generate_empty_sudoku(0.3)


[[0, 0, 4, 0, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 5, 2, 9, 6, 0],
 [2, 0, 0, 0, 3, 7, 0, 0, 6],
 [0, 0, 0, 7, 0, 0, 0, 0, 0],
 [0, 3, 0, 0, 2, 3, 0, 6, 0],
 [0, 0, 0, 7, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 2, 0, 0, 0, 0],
 [0, 9, 5, 0, 4, 0, 0, 5, 0],
 [0, 0, 9, 0, 0, 0, 1, 0, 0]]

#  genereren van een lege twodoku met % van c random gevulde cellen

In [61]:
from typing import List, Tuple
import random

Grid = List[List[int]]

def format_grid(grid: Grid) -> str:
    return "\n".join(" ".join(f"{v:2d}" for v in row) for row in grid)

def generate_twodoku_sudoku(
    filled_percentage: float,
    rows: int,
    columns: int,
    sudoku_one: tuple[int,int],
    sudoku_two: tuple[int,int]) -> Grid:

    # Start with empty grid
    grid: Grid = [[0 for _ in range(columns)] for _ in range(rows)]

    # Determine which cells actually belong to at least one 9x9 block
    twodoku_cells = set()
    twodoku_blocks = [sudoku_one, sudoku_two]
    for top, left in twodoku_blocks:
        for r in range(9):
            for c in range(9):
                rr = top + r
                cc = left + c
                if 0 <= rr < rows and 0 <= cc < columns:
                    twodoku_cells.add((rr, cc))

    twodoku_cells = list(twodoku_cells)
    total_valid = len(twodoku_cells)
    cells_to_fill = int(total_valid * filled_percentage)

    # Fill random cells with clues
    random.shuffle(twodoku_cells)
    cells_to_fill_positions = twodoku_cells[:cells_to_fill]

    for (r, c) in cells_to_fill_positions:
        grid[r][c] = random.randint(1, 9)
    
    sudoku_set = set(twodoku_cells)
    for r in range(rows):
        for c in range(columns):
            if (r, c) not in sudoku_set:
                grid[r][c] = -1
    return grid

## Negen bij negen sudoku's met drie bij drie overlap

In [ ]:
NineByNine = generate_twodoku_sudoku(filled_percentage=0.5, 
                                  rows=15, 
                                  columns=15, 
                                  sudoku_one=(0, 0), 
                                  sudoku_two=(6, 6))

# Function for nicer printing
print(format_grid(NineByNine))

def grid_to_string(grid):
    return "\n".join(
        " ".join(str(x) for x in row)
        for row in grid)

twodoku_str = grid_to_string(NineByNine)

def write_grid_to_file(grid, path):
    with open(path, "w") as f:
        for row in grid:
            line = " ".join(str(x) for x in row)
            f.write(line + "\n")

write_grid_to_file(NineByNine, "twodoku_example.txt")

 0  9  0  0  0  5  4  0  2 -1 -1 -1 -1 -1 -1
 3  9  8  0  2  2  7  2  9 -1 -1 -1 -1 -1 -1
 6  0  0  1  3  8  0  8  2 -1 -1 -1 -1 -1 -1
 0  8  0  0  0  3  0  0  1 -1 -1 -1 -1 -1 -1
 9  0  4  0  0  0  0  0  3 -1 -1 -1 -1 -1 -1
 4  9  6  4  0  0  0  7  0 -1 -1 -1 -1 -1 -1
 8  5  0  5  0  6  0  0  8  0  9  0  0  4  2
 4  9  0  8  0  0  9  0  0  0  0  9  0  0  0
 0  2  0  0  0  0  1  0  0  7  0  7  4  0  3
-1 -1 -1 -1 -1 -1  5  0  0  8  8  0  6  0  7
-1 -1 -1 -1 -1 -1  3  8  0  0  0  5  9  5  0
-1 -1 -1 -1 -1 -1  2  0  4  0  8  0  0  2  9
-1 -1 -1 -1 -1 -1  0  0  0  9  1  1  0  3  0
-1 -1 -1 -1 -1 -1  0  5  4  4  1  6  4  0  0
-1 -1 -1 -1 -1 -1  8  0  2  0  0  0  9  0  0


# Encoden van twodoku's met -1 cellen

In [ ]:
import math

def read_grid(path: str) -> list[list[int]]:
    with open(path) as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]
    grid: list[list[int]] = []
    for line in lines:
        grid.append([int(x) for x in line.split()])
    return grid

def to_cnf(input_path: str):
    grid = read_grid(input_path)
    R = len(grid)
    C = len(grid[0])
    # Sudoku digits 1..9 (you can parameterize if you want)
    N = 9

    # active[r][c] = True iff this cell is part of (some) Sudoku
    active = [[grid[r][c] != -1 for c in range(C)] for r in range(R)]

    clauses = []
    clauses.extend(exactly_one_v_per_cell(R, C, N, active))
    clauses.extend(row_constraint(R, C, N, active))
    clauses.extend(column_constraint(R, C, N, active))
    clauses.extend(box_constraint(R, C, N, active))
    clauses.extend(orthogonal_constraint(R, C, N, active))
    clauses.extend(clues_constraint(grid, R, C, N, active))

    # variable mapping still uses the *physical* grid size R,C,
    # but num_vars you can just set to R*C*N or, if your assignment
    # insists, max var you've used.
    num_vars = R * C * N

    clauses = check_for_duplicates(clauses)
    return clauses, num_vars

def check_for_duplicates(clauses):
    """
    Check for duplicate clauses and remove to save space
    """
    duplicate_clauses = set()
    unique_clauses = []
    for c in clauses:
        clause = tuple(sorted(c))
        if clause not in duplicate_clauses:
            duplicate_clauses.add(clause)
            unique_clauses.append(list(clause))
    return unique_clauses

def var_mapping(r: int, c: int, v: int, N: int, C: int) -> int:
    # r in 0..R-1, c in 0..C-1, v in 1..N
    return r * C * N + c * N + v

def exactly_one_v_per_cell(R, C, N, active):
    clauses = []

    # at least one
    for r in range(R):
        for c in range(C):
            if not active[r][c]:
                continue
            clause = [var_mapping(r, c, v, N, C) for v in range(1, N+1)]
            clauses.append(clause)

    # at most one
    for r in range(R):
        for c in range(C):
            if not active[r][c]:
                continue
            for v1 in range(1, N):
                for v2 in range(v1+1, N+1):
                    v1_id = var_mapping(r, c, v1, N, C)
                    v2_id = var_mapping(r, c, v2, N, C)
                    clauses.append([-v1_id, -v2_id])

    return clauses

def row_constraint(R, C, N, active):
    clauses = []
    for r in range(R):
        for v in range(1, N+1):
            cells = [c for c in range(C) if active[r][c]]
            if not cells:
                continue
            # at least one
            clauses.append([var_mapping(r, c, v, N, C) for c in cells])
            # at most one
            for i in range(len(cells) - 1):
                for j in range(i+1, len(cells)):
                    c1, c2 = cells[i], cells[j]
                    x = var_mapping(r, c1, v, N, C)
                    y = var_mapping(r, c2, v, N, C)
                    clauses.append([-x, -y])
    return clauses

def column_constraint(R, C, N, active):
    clauses = []
    for c in range(C):
        for v in range(1, N+1):
            cells = [r for r in range(R) if active[r][c]]
            if not cells:
                continue
            # at least one
            clauses.append([var_mapping(r, c, v, N, C) for r in cells])
            # at most one
            for i in range(len(cells) - 1):
                for j in range(i+1, len(cells)):
                    r1, r2 = cells[i], cells[j]
                    x = var_mapping(r1, c, v, N, C)
                    y = var_mapping(r2, c, v, N, C)
                    clauses.append([-x, -y])
    return clauses

def row_constraint(R, C, N, active):
    clauses = []
    for r in range(R):
        for v in range(1, N+1):
            cells = [c for c in range(C) if active[r][c]]
            if not cells:
                continue
            # at least one
            clauses.append([var_mapping(r, c, v, N, C) for c in cells])
            # at most one
            for i in range(len(cells) - 1):
                for j in range(i+1, len(cells)):
                    c1, c2 = cells[i], cells[j]
                    x = var_mapping(r, c1, v, N, C)
                    y = var_mapping(r, c2, v, N, C)
                    clauses.append([-x, -y])
    return clauses

def column_constraint(R, C, N, active):
    clauses = []
    for c in range(C):
        for v in range(1, N+1):
            cells = [r for r in range(R) if active[r][c]]
            if not cells:
                continue
            # at least one
            clauses.append([var_mapping(r, c, v, N, C) for r in cells])
            # at most one
            for i in range(len(cells) - 1):
                for j in range(i+1, len(cells)):
                    r1, r2 = cells[i], cells[j]
                    x = var_mapping(r1, c, v, N, C)
                    y = var_mapping(r2, c, v, N, C)
                    clauses.append([-x, -y])
    return clauses

def box_constraint(R, C, N, active, boards=None):
    if boards is None:
        boards = [(0, 0)]  # default: single 9x9 at top-left

    B = int(math.sqrt(N))  # 3 for standard Sudoku
    clauses = []

    for (top, left) in boards:
        # iterate 3x3 boxes inside this 9x9
        for br in range(0, N, B):
            for bc in range(0, N, B):
                # collect cells inside this box that are active
                cells = []
                for dr in range(B):
                    for dc in range(B):
                        r = top + br + dr
                        c = left + bc + dc
                        if 0 <= r < R and 0 <= c < C and active[r][c]:
                            cells.append((r, c))
                if not cells:
                    continue
                # Sudoku "exactly one v per box"
                for v in range(1, N+1):
                    # at least one
                    clauses.append([var_mapping(r, c, v, N, C) for (r, c) in cells])
                    # at most one
                    for i in range(len(cells)-1):
                        for j in range(i+1, len(cells)):
                            r1, c1 = cells[i]
                            r2, c2 = cells[j]
                            x = var_mapping(r1, c1, v, N, C)
                            y = var_mapping(r2, c2, v, N, C)
                            clauses.append([-x, -y])
    return clauses

def orthogonal_constraint(R, C, N, active):
    clauses = []
    directions = [(0, 1), (1, 0)]  # right, down
    for r in range(R):
        for c in range(C):
            if not active[r][c]:
                continue
            for dr, dc in directions:
                r2, c2 = r + dr, c + dc
                if 0 <= r2 < R and 0 <= c2 < C and active[r2][c2]:
                    for v in range(1, N):
                        x1 = var_mapping(r, c, v, N, C)
                        x2 = var_mapping(r2, c2, v+1, N, C)
                        y1 = var_mapping(r, c, v+1, N, C)
                        y2 = var_mapping(r2, c2, v, N, C)
                        clauses.append([-x1, -x2])
                        clauses.append([-y1, -y2])
    return clauses

def clues_constraint(grid, R, C, N, active):
    clauses = []
    for r in range(R):
        for c in range(C):
            if not active[r][c]:
                continue
            v = grid[r][c]
            if 1 <= v <= N:  # ignore 0 and -1
                var = var_mapping(r, c, v, N, C)
                clauses.append([var])
    return clauses

In [79]:
to_cnf("twodoku_example.txt")

([[1, 2, 3, 4, 5, 6, 7, 8, 9],
  [10, 11, 12, 13, 14, 15, 16, 17, 18],
  [19, 20, 21, 22, 23, 24, 25, 26, 27],
  [28, 29, 30, 31, 32, 33, 34, 35, 36],
  [37, 38, 39, 40, 41, 42, 43, 44, 45],
  [46, 47, 48, 49, 50, 51, 52, 53, 54],
  [55, 56, 57, 58, 59, 60, 61, 62, 63],
  [64, 65, 66, 67, 68, 69, 70, 71, 72],
  [73, 74, 75, 76, 77, 78, 79, 80, 81],
  [136, 137, 138, 139, 140, 141, 142, 143, 144],
  [145, 146, 147, 148, 149, 150, 151, 152, 153],
  [154, 155, 156, 157, 158, 159, 160, 161, 162],
  [163, 164, 165, 166, 167, 168, 169, 170, 171],
  [172, 173, 174, 175, 176, 177, 178, 179, 180],
  [181, 182, 183, 184, 185, 186, 187, 188, 189],
  [190, 191, 192, 193, 194, 195, 196, 197, 198],
  [199, 200, 201, 202, 203, 204, 205, 206, 207],
  [208, 209, 210, 211, 212, 213, 214, 215, 216],
  [271, 272, 273, 274, 275, 276, 277, 278, 279],
  [280, 281, 282, 283, 284, 285, 286, 287, 288],
  [289, 290, 291, 292, 293, 294, 295, 296, 297],
  [298, 299, 300, 301, 302, 303, 304, 305, 306],
  [307, 308,

In [ ]:
import argparse
import sys
from encoder import to_cnf  


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()


def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--in", dest="inp", required=True, help="Path to puzzle .txt")
    p.add_argument("--out", dest="out", default=None, help="Path to write DIMACS CNF (stdout if omitted)")
    return p.parse_args()


def main():
    args = parse_args()
    clauses, num_vars = to_cnf(args.inp)
    if args.out:
        write_dimacs(args.out, num_vars, clauses)
    else:
        write_dimacs(sys.stdout, num_vars, clauses)


if __name__ == "__main__":
    main()
    # args = parse_args()
    # grid = generate_twodoku_sudoku(
    #     filled_percentage=args.filled_percentage,
    #     rows=args.rows,
    #     columns=args.columns,
    #     sudoku_one=args.sudoku_one,
    #     sudoku_two=args.sudoku_two)

    # output_str = grid_to_string(grid)

    # if args.out:
    #     with open(args.out, "w") as f:
    #         f.write(output_str + "\n")
    # else:
    #     print(output_str)

# Mainpy met oude manier van genereren generator.py

In [ ]:
import argparse
import sys
import os
from encoder import to_cnf  
from generator import generate_twodoku_sudoku, grid_to_string

def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()
def parse_tuple(tuple):
    try:
        a, b = tuple.split(",")
        return int(a), int(b)
    except:
        raise argparse.ArgumentTypeError("Tuples must be in format: x,y")

def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--filled_percentage", "-f",
                    type=float,
                    default=0.5)
    p.add_argument("--rows", "-r",
                    type=int,
                    default=15)
    p.add_argument("--columns", "-c",
                    type=int,
                    default=15)
    p.add_argument("--sudoku_one", "-s1",
                    type=parse_tuple,
                    default=(0, 0))
    p.add_argument("--sudoku_two", "-s2",
                    type=parse_tuple,
                    default=(6, 6))
    p.add_argument("--out", 
                   dest="out", 
                   default=None, 
                   help="Path to write DIMACS CNF (stdout if omitted)")
    return p.parse_args()
"""
def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--in", dest="inp", required=True, help="Path to puzzle .txt")
    p.add_argument("--out", dest="out", default=None, help="Path to write DIMACS CNF (stdout if omitted)")
    return p.parse_args()"""


def main():
    # Generate a TwoDoku puzzle based on command line arguments
    args = parse_args()
    puzzles = []
    
    # Generate 20 files
    for i in range(1, 21):
        grid = generate_twodoku_sudoku(
            filled_percentage=args.filled_percentage,
            rows=args.rows,
            columns=args.columns,
            sudoku_one=args.sudoku_one,
            sudoku_two=args.sudoku_two)

        output_str = grid_to_string(grid)

        filename = f"{"TwoDoku"}_{i}{".txt"}"
        with open(filename, "w") as f:
            f.write(output_str + "\n")
        
        puzzles.append(filename)

    # Encode clauses to CNF and write to DIMACS format
    for puzzle_path in puzzles:
        clauses, num_vars = to_cnf(puzzle_path)

        # Same base name as puzzle, but .cnf extension
        cnf_path = os.path.splitext(puzzle_path)[0] + ".cnf"
        write_dimacs(cnf_path, num_vars, clauses)

    # Solve the TwoDoku puzzle










if __name__ == "__main__":
    main()